## Set Up Environment and Upload Data

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

from IPython.core.display import HTML
from matplotlib.pyplot import figure
from os import listdir
from os.path import isfile, join
import re
import sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples
from matplotlib.ticker import FixedLocator, FixedFormatter
import matplotlib as mpl
from scipy.stats import zscore
from scipy.spatial import distance_matrix

pd.set_option('display.max_columns', None)

In [51]:
path = 'C:\\Users\\steve\\OneDrive\\Desktop\\Github\\NFL Big Data Bowl\\0. Data'
allFiles = [f for f in listdir(path) if isfile(join(path, f))]

In [52]:
# Upload files
games = pd.read_csv("../0. Data/games.csv")
players = pd.read_csv("../0. Data/players.csv")
plays = pd.read_csv("../0. Data/plays.csv")

fbf_df = pd.DataFrame()

# Go through all weekly data and upload it
for file in allFiles:
    if "week" in file:
        temp_week = pd.read_csv("../0. Data/"+file)
        temp_week['week'] = re.findall(r'\d+', file)[0]
        fbf_df = pd.concat([fbf_df, temp_week])

# merge with games to add home/away team indicators
fbf_df = fbf_df.merge(games[['gameId','homeTeamAbbr','visitorTeamAbbr']], on='gameId')

In [53]:
# fbf_df_original = fbf_df.copy()

In [75]:
fbf_df = fbf_df_original.copy()

## Data Exploration

In [4]:
plays.head()

gameId  playId                                    playDescription  \
0  2018090600      75  (15:00) M.Ryan pass short right to J.Jones pus...   
1  2018090600     146  (13:10) M.Ryan pass incomplete short right to ...   
2  2018090600     168  (13:05) (Shotgun) M.Ryan pass incomplete short...   
3  2018090600     190  (13:01) (Shotgun) M.Ryan pass deep left to J.J...   
4  2018090600     256  (10:59) (Shotgun) M.Ryan pass incomplete short...   

   quarter  down  yardsToGo possessionTeam        playType yardlineSide  \
0        1     1         15            ATL  play_type_pass          ATL   
1        1     1         10            ATL  play_type_pass          PHI   
2        1     2         10            ATL  play_type_pass          PHI   
3        1     3         10            ATL  play_type_pass          PHI   
4        1     3          1            ATL  play_type_pass          PHI   

   yardlineNumber offenseFormation        personnelO  defendersInTheBox  \
0              20           I_FORM  2 RB, 1 TE, 2 WR                7.0   
1              39       SINGLEBACK  1 RB, 1 TE, 3 WR                7.0   
2              39          SHOTGUN  2 RB, 1 TE, 2 WR                6.0   
3              39          SHOTGUN  1 RB, 1 TE, 3 WR                6.0   
4               1          SHOTGUN  2 RB, 3 TE, 0 WR                8.0   

   numberOfPassRushers        personnelD           typeDropback  \
0                  4.0  4 DL, 2 LB, 5 DB            TRADITIONAL   
1                  4.0  4 DL, 2 LB, 5 DB            TRADITIONAL   
2                  4.0  4 DL, 2 LB, 5 DB            TRADITIONAL   
3                  5.0  4 DL, 1 LB, 6 DB  SCRAMBLE_ROLLOUT_LEFT   
4                  6.0  6 DL, 3 LB, 2 DB            TRADITIONAL   

   preSnapVisitorScore  preSnapHomeScore gameClock  absoluteYardlineNumber  \
0                  0.0               0.0  15:00:00                    90.0   
1                  0.0               0.0  13:10:00                    49.0   
2                  0.0               0.0  13:05:00                    49.0   
3                  0.0               0.0  13:01:00                    49.0   
4                  0.0               0.0  10:59:00                    11.0   

  penaltyCodes penaltyJerseyNumbers passResult  offensePlayResult  playResult  \
0          NaN                  NaN          C                 10          10   
1          NaN                  NaN          I                  0           0   
2          NaN                  NaN          I                  0           0   
3          NaN                  NaN          C                 33          33   
4          NaN                  NaN          I                  0           0   

        epa  isDefensivePI  
0  0.261827          False  
1 -0.372360          False  
2 -0.702779          False  
3  3.047530          False  
4 -0.842272          False

In [5]:
plays.columns

Index(['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'playType', 'yardlineSide', 'yardlineNumber',
       'offenseFormation', 'personnelO', 'defendersInTheBox',
       'numberOfPassRushers', 'personnelD', 'typeDropback',
       'preSnapVisitorScore', 'preSnapHomeScore', 'gameClock',
       'absoluteYardlineNumber', 'penaltyCodes', 'penaltyJerseyNumbers',
       'passResult', 'offensePlayResult', 'playResult', 'epa',
       'isDefensivePI'],
      dtype='object')

In [6]:
plays.playType.value_counts()

play_type_pass       17346
play_type_sack        1260
play_type_unknown      633
Name: playType, dtype: int64

In [175]:
filtered_plays = plays[plays['playType']=='play_type_pass']

In [176]:
filtered_plays.offenseFormation.value_counts()

SHOTGUN       11364
SINGLEBACK     2556
EMPTY          2209
I_FORM          829
PISTOL          217
JUMBO            46
WILDCAT          31
Name: offenseFormation, dtype: int64

In [177]:
filtered_plays.isDefensivePI.value_counts()

False    17324
True        22
Name: isDefensivePI, dtype: int64

In [178]:
filtered_plays = filtered_plays[filtered_plays['isDefensivePI']==False]

In [179]:
filtered_plays.head()

gameId  playId                                    playDescription  \
0  2018090600      75  (15:00) M.Ryan pass short right to J.Jones pus...   
1  2018090600     146  (13:10) M.Ryan pass incomplete short right to ...   
2  2018090600     168  (13:05) (Shotgun) M.Ryan pass incomplete short...   
3  2018090600     190  (13:01) (Shotgun) M.Ryan pass deep left to J.J...   
4  2018090600     256  (10:59) (Shotgun) M.Ryan pass incomplete short...   

   quarter  down  yardsToGo possessionTeam        playType yardlineSide  \
0        1     1         15            ATL  play_type_pass          ATL   
1        1     1         10            ATL  play_type_pass          PHI   
2        1     2         10            ATL  play_type_pass          PHI   
3        1     3         10            ATL  play_type_pass          PHI   
4        1     3          1            ATL  play_type_pass          PHI   

   yardlineNumber offenseFormation        personnelO  defendersInTheBox  \
0              20           I_FORM  2 RB, 1 TE, 2 WR                7.0   
1              39       SINGLEBACK  1 RB, 1 TE, 3 WR                7.0   
2              39          SHOTGUN  2 RB, 1 TE, 2 WR                6.0   
3              39          SHOTGUN  1 RB, 1 TE, 3 WR                6.0   
4               1          SHOTGUN  2 RB, 3 TE, 0 WR                8.0   

   numberOfPassRushers        personnelD           typeDropback  \
0                  4.0  4 DL, 2 LB, 5 DB            TRADITIONAL   
1                  4.0  4 DL, 2 LB, 5 DB            TRADITIONAL   
2                  4.0  4 DL, 2 LB, 5 DB            TRADITIONAL   
3                  5.0  4 DL, 1 LB, 6 DB  SCRAMBLE_ROLLOUT_LEFT   
4                  6.0  6 DL, 3 LB, 2 DB            TRADITIONAL   

   preSnapVisitorScore  preSnapHomeScore gameClock  absoluteYardlineNumber  \
0                  0.0               0.0  15:00:00                    90.0   
1                  0.0               0.0  13:10:00                    49.0   
2                  0.0               0.0  13:05:00                    49.0   
3                  0.0               0.0  13:01:00                    49.0   
4                  0.0               0.0  10:59:00                    11.0   

  penaltyCodes penaltyJerseyNumbers passResult  offensePlayResult  playResult  \
0          NaN                  NaN          C                 10          10   
1          NaN                  NaN          I                  0           0   
2          NaN                  NaN          I                  0           0   
3          NaN                  NaN          C                 33          33   
4          NaN                  NaN          I                  0           0   

        epa  isDefensivePI  
0  0.261827          False  
1 -0.372360          False  
2 -0.702779          False  
3  3.047530          False  
4 -0.842272          False

In [77]:
fbf_df = fbf_df.merge(plays[['possessionTeam','gameId','playId']], on=['gameId','playId'], how='left')

## Find Closest Defender

In [165]:
fbf_df_test = fbf_df.copy()

In [167]:
fbf_df_test['team'] = np.where(fbf_df_test['team']=='home',fbf_df_test['homeTeamAbbr'],
                                 fbf_df_test['visitorTeamAbbr'])
fbf_df_test["inPoss"] = np.where(fbf_df_test['possessionTeam']==fbf_df_test['team'],1,0)

In [78]:
fbf_df.head()

time      x      y     s     a   dis       o     dir  \
0  2018-09-07T01:07:14.599Z  91.73  26.67  0.00  0.01  0.02  289.57  240.93   
1  2018-09-07T01:07:14.599Z  88.89  36.47  0.01  0.01  0.01  105.63   66.66   
2  2018-09-07T01:07:14.599Z  91.35  44.16  0.02  0.03  0.01  290.45   16.86   
3  2018-09-07T01:07:14.599Z  86.31  22.01  0.09  0.42  0.01   70.12  168.91   
4  2018-09-07T01:07:14.599Z  90.78  36.15  0.00  0.00  0.00  257.61  193.97   

  event      nflId      displayName  jerseyNumber position  frameId  team  \
0  None      310.0        Matt Ryan           2.0       QB        1  away   
1  None    79848.0  Malcolm Jenkins          27.0       SS        1  home   
2  None  2495454.0      Julio Jones          11.0       WR        1  away   
3  None  2495613.0     Corey Graham          24.0       FS        1  home   
4  None  2533040.0     Mohamed Sanu          12.0       WR        1  away   

       gameId  playId playDirection  route week homeTeamAbbr visitorTeamAbbr  \
0  2018090600      75          left    NaN    1          PHI             ATL   
1  2018090600      75          left    NaN    1          PHI             ATL   
2  2018090600      75          left  HITCH    1          PHI             ATL   
3  2018090600      75          left    NaN    1          PHI             ATL   
4  2018090600      75          left  HITCH    1          PHI             ATL   

  possessionTeam  
0            ATL  
1            ATL  
2            ATL  
3            ATL  
4            ATL

In [184]:
fbf_df_sample = fbf_df[(fbf_df.playId==75)&(fbf_df.gameId==2018090600)]

In [185]:
fbf_df_sample['team'] = np.where(fbf_df_sample['team']=='home',fbf_df_sample['homeTeamAbbr'],
                                 fbf_df_sample['visitorTeamAbbr'])
fbf_df_sample["inPoss"] = np.where(fbf_df_sample['possessionTeam']==fbf_df_sample['team'],1,0)

<ipython-input-185-e6b473a7dae5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fbf_df_sample['team'] = np.where(fbf_df_sample['team']=='home',fbf_df_sample['homeTeamAbbr'],
<ipython-input-185-e6b473a7dae5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fbf_df_sample["inPoss"] = np.where(fbf_df_sample['possessionTeam']==fbf_df_sample['team'],1,0)


In [201]:
fbf_df_sample_frame = fbf_df_sample[fbf_df_sample['frameId']==60]

In [202]:
fbf_df_sample_frame_WR = fbf_df_sample_frame[(fbf_df_sample_frame["inPoss"]==1)&(fbf_df_sample_frame['position']=="WR")]
fbf_df_sample_frame_D =  fbf_df_sample_frame[fbf_df_sample_frame["inPoss"]==0]

In [203]:
fbf_df_sample_frame_WR

Empty DataFrame
Columns: [time, x, y, s, a, dis, o, dir, event, nflId, displayName, jerseyNumber, position, frameId, team, gameId, playId, playDirection, route, week, homeTeamAbbr, visitorTeamAbbr, possessionTeam, inPoss]
Index: []

In [204]:
fbf_df_sample_frame_D

Empty DataFrame
Columns: [time, x, y, s, a, dis, o, dir, event, nflId, displayName, jerseyNumber, position, frameId, team, gameId, playId, playDirection, route, week, homeTeamAbbr, visitorTeamAbbr, possessionTeam, inPoss]
Index: []

In [205]:
fbf_df_sample_frame

Empty DataFrame
Columns: [time, x, y, s, a, dis, o, dir, event, nflId, displayName, jerseyNumber, position, frameId, team, gameId, playId, playDirection, route, week, homeTeamAbbr, visitorTeamAbbr, possessionTeam, inPoss]
Index: []

In [191]:
dm = distance_matrix(fbf_df_sample_frame_WR[['x','y']].values, 
                     fbf_df_sample_frame_D[['x','y']].values)

In [192]:
dm.shape

(2, 7)

In [193]:
dm

array([[ 1.13600176, 32.92231006, 14.55742079, 12.50313561, 42.56472013,
         3.79110802, 16.45223389],
       [15.75967639, 16.87647475,  4.78357607,  5.39967592, 25.8952988 ,
        13.98005007,  1.84219977]])

In [194]:
# dm[dm==0] = np.inf

In [195]:
closest_defender = dm.argmin(axis=1)

In [196]:
closest_defender_distance = dm.min(axis=1)

In [197]:
closest_defender_distance

array([1.13600176, 1.84219977])

In [198]:
closest_defender

array([0, 6], dtype=int64)

In [199]:
for i,j in enumerate(closest_defender):
    fbf_df_sample_frame_WR.displayName.iloc[i]
    fbf_df_sample_frame_D.displayName.iloc[j]
    

In [207]:
temp = pd.DataFrame()
for game_id in fbf_df_test.gameId.unique():
    for play_id in fbf_df_test[fbf_df_test.gameId==game_id].playId.unique():
        for frame_id in fbf_df_test[(fbf_df_test.playId==play_id)&(fbf_df_test.gameId==game_id)].frameId.unique():
            print("Game: {} | Play: {} | Frame: {}".format(game_id,play_id,frame_id))
            df = fbf_df_test[(fbf_df_test.gameId==game_id)&
                    (fbf_df_test.playId==play_id)&
                    (fbf_df_test.frameId==frame_id)
                   ]

            df_wr = df[(df["inPoss"]==1)&(df['position']=="WR")]
            df_d =  df[df["inPoss"]==0]
            dm = distance_matrix(df_wr[['x','y']].values, 
                     df_d[['x','y']].values)
            closest_defender = dm.argmin(axis=1)
            closest_defender_distance = dm.min(axis=1)

            for i,j in enumerate(closest_defender):
                temp_df = pd.DataFrame({
                    'gameId':[game_id]
                    ,'playId':[play_id]
                    ,'frameId':[frame_id]
                    ,'displayName':[df_wr.displayName.iloc[i]]
                    ,'closestDefender':[df_d.displayName.iloc[j]]
                    ,'closestDefenderDistance':[closest_defender_distance[i]]
                })
                temp = pd.concat([temp, temp_df])

Game: 2018090600 | Play: 75 | Frame: 1
Game: 2018090600 | Play: 75 | Frame: 2
Game: 2018090600 | Play: 75 | Frame: 3
Game: 2018090600 | Play: 75 | Frame: 4
Game: 2018090600 | Play: 75 | Frame: 5
Game: 2018090600 | Play: 75 | Frame: 6
Game: 2018090600 | Play: 75 | Frame: 7
Game: 2018090600 | Play: 75 | Frame: 8
Game: 2018090600 | Play: 75 | Frame: 9
Game: 2018090600 | Play: 75 | Frame: 10
Game: 2018090600 | Play: 75 | Frame: 11
Game: 2018090600 | Play: 75 | Frame: 12
Game: 2018090600 | Play: 75 | Frame: 13
Game: 2018090600 | Play: 75 | Frame: 14
Game: 2018090600 | Play: 75 | Frame: 15
Game: 2018090600 | Play: 75 | Frame: 16
Game: 2018090600 | Play: 75 | Frame: 17
Game: 2018090600 | Play: 75 | Frame: 18
Game: 2018090600 | Play: 75 | Frame: 19
Game: 2018090600 | Play: 75 | Frame: 20
Game: 2018090600 | Play: 75 | Frame: 21
Game: 2018090600 | Play: 75 | Frame: 22
Game: 2018090600 | Play: 75 | Frame: 23
Game: 2018090600 | Play: 75 | Frame: 24
Game: 2018090600 | Play: 75 | Frame: 25
Game: 201

Game: 2018090600 | Play: 190 | Frame: 49
Game: 2018090600 | Play: 190 | Frame: 50
Game: 2018090600 | Play: 190 | Frame: 51
Game: 2018090600 | Play: 190 | Frame: 52
Game: 2018090600 | Play: 190 | Frame: 53
Game: 2018090600 | Play: 190 | Frame: 54
Game: 2018090600 | Play: 190 | Frame: 55
Game: 2018090600 | Play: 190 | Frame: 56
Game: 2018090600 | Play: 190 | Frame: 57
Game: 2018090600 | Play: 190 | Frame: 58
Game: 2018090600 | Play: 190 | Frame: 59
Game: 2018090600 | Play: 190 | Frame: 60
Game: 2018090600 | Play: 190 | Frame: 61
Game: 2018090600 | Play: 190 | Frame: 62
Game: 2018090600 | Play: 190 | Frame: 63
Game: 2018090600 | Play: 190 | Frame: 64
Game: 2018090600 | Play: 190 | Frame: 65
Game: 2018090600 | Play: 190 | Frame: 66
Game: 2018090600 | Play: 190 | Frame: 67
Game: 2018090600 | Play: 190 | Frame: 68
Game: 2018090600 | Play: 190 | Frame: 69
Game: 2018090600 | Play: 190 | Frame: 70
Game: 2018090600 | Play: 190 | Frame: 71
Game: 2018090600 | Play: 190 | Frame: 72
Game: 2018090600

Game: 2018090600 | Play: 402 | Frame: 34
Game: 2018090600 | Play: 402 | Frame: 35
Game: 2018090600 | Play: 402 | Frame: 36
Game: 2018090600 | Play: 402 | Frame: 37
Game: 2018090600 | Play: 402 | Frame: 38
Game: 2018090600 | Play: 402 | Frame: 39
Game: 2018090600 | Play: 402 | Frame: 40
Game: 2018090600 | Play: 402 | Frame: 41
Game: 2018090600 | Play: 402 | Frame: 42
Game: 2018090600 | Play: 402 | Frame: 43
Game: 2018090600 | Play: 402 | Frame: 44
Game: 2018090600 | Play: 402 | Frame: 45
Game: 2018090600 | Play: 402 | Frame: 46
Game: 2018090600 | Play: 402 | Frame: 47
Game: 2018090600 | Play: 402 | Frame: 48
Game: 2018090600 | Play: 402 | Frame: 49
Game: 2018090600 | Play: 402 | Frame: 50
Game: 2018090600 | Play: 402 | Frame: 51
Game: 2018090600 | Play: 402 | Frame: 52
Game: 2018090600 | Play: 402 | Frame: 53
Game: 2018090600 | Play: 402 | Frame: 54
Game: 2018090600 | Play: 402 | Frame: 55
Game: 2018090600 | Play: 402 | Frame: 56
Game: 2018090600 | Play: 402 | Frame: 57
Game: 2018090600

Game: 2018090600 | Play: 545 | Frame: 13
Game: 2018090600 | Play: 545 | Frame: 14
Game: 2018090600 | Play: 545 | Frame: 15
Game: 2018090600 | Play: 545 | Frame: 16
Game: 2018090600 | Play: 545 | Frame: 17
Game: 2018090600 | Play: 545 | Frame: 18
Game: 2018090600 | Play: 545 | Frame: 19
Game: 2018090600 | Play: 545 | Frame: 20
Game: 2018090600 | Play: 545 | Frame: 21
Game: 2018090600 | Play: 545 | Frame: 22
Game: 2018090600 | Play: 545 | Frame: 23
Game: 2018090600 | Play: 545 | Frame: 24
Game: 2018090600 | Play: 545 | Frame: 25
Game: 2018090600 | Play: 545 | Frame: 26
Game: 2018090600 | Play: 545 | Frame: 27
Game: 2018090600 | Play: 545 | Frame: 28
Game: 2018090600 | Play: 545 | Frame: 29
Game: 2018090600 | Play: 545 | Frame: 30
Game: 2018090600 | Play: 545 | Frame: 31
Game: 2018090600 | Play: 545 | Frame: 32
Game: 2018090600 | Play: 545 | Frame: 33
Game: 2018090600 | Play: 545 | Frame: 34
Game: 2018090600 | Play: 545 | Frame: 35
Game: 2018090600 | Play: 545 | Frame: 36
Game: 2018090600

Game: 2018090600 | Play: 776 | Frame: 21
Game: 2018090600 | Play: 776 | Frame: 22
Game: 2018090600 | Play: 776 | Frame: 23
Game: 2018090600 | Play: 776 | Frame: 24
Game: 2018090600 | Play: 776 | Frame: 25
Game: 2018090600 | Play: 776 | Frame: 26
Game: 2018090600 | Play: 776 | Frame: 27
Game: 2018090600 | Play: 776 | Frame: 28
Game: 2018090600 | Play: 776 | Frame: 29
Game: 2018090600 | Play: 776 | Frame: 30
Game: 2018090600 | Play: 776 | Frame: 31
Game: 2018090600 | Play: 776 | Frame: 32
Game: 2018090600 | Play: 776 | Frame: 33
Game: 2018090600 | Play: 776 | Frame: 34
Game: 2018090600 | Play: 776 | Frame: 35
Game: 2018090600 | Play: 776 | Frame: 36
Game: 2018090600 | Play: 776 | Frame: 37
Game: 2018090600 | Play: 776 | Frame: 38
Game: 2018090600 | Play: 776 | Frame: 39
Game: 2018090600 | Play: 776 | Frame: 40
Game: 2018090600 | Play: 776 | Frame: 41
Game: 2018090600 | Play: 776 | Frame: 42
Game: 2018090600 | Play: 776 | Frame: 43
Game: 2018090600 | Play: 776 | Frame: 44
Game: 2018090600

...

Game: 2018090906 | Play: 3363 | Frame: 39
Game: 2018090906 | Play: 3363 | Frame: 40
Game: 2018090906 | Play: 3363 | Frame: 41
Game: 2018090906 | Play: 3363 | Frame: 42
Game: 2018090906 | Play: 3363 | Frame: 43
Game: 2018090906 | Play: 3363 | Frame: 44
Game: 2018090906 | Play: 3363 | Frame: 45
Game: 2018090906 | Play: 3363 | Frame: 46
Game: 2018090906 | Play: 3363 | Frame: 47
Game: 2018090906 | Play: 3363 | Frame: 48
Game: 2018090906 | Play: 3363 | Frame: 49
Game: 2018090906 | Play: 3363 | Frame: 50
Game: 2018090906 | Play: 3363 | Frame: 51
Game: 2018090906 | Play: 3363 | Frame: 52
Game: 2018090906 | Play: 3363 | Frame: 53
Game: 2018090906 | Play: 3363 | Frame: 54
Game: 2018090906 | Play: 3363 | Frame: 55
Game: 2018090906 | Play: 3363 | Frame: 56
Game: 2018090906 | Play: 3363 | Frame: 57
Game: 2018090906 | Play: 3363 | Frame: 58
Game: 2018090906 | Play: 3363 | Frame: 59
Game: 2018090906 | Play: 3363 | Frame: 60
Game: 2018090906 | Play: 3363 | Frame: 61
Game: 2018090906 | Play: 3363 | Fr

Game: 2018090906 | Play: 3546 | Frame: 41
Game: 2018090906 | Play: 3546 | Frame: 42
Game: 2018090906 | Play: 3546 | Frame: 43
Game: 2018090906 | Play: 3546 | Frame: 44
Game: 2018090906 | Play: 3546 | Frame: 45
Game: 2018090906 | Play: 3546 | Frame: 46
Game: 2018090906 | Play: 3546 | Frame: 47
Game: 2018090906 | Play: 3546 | Frame: 48
Game: 2018090906 | Play: 3546 | Frame: 49
Game: 2018090906 | Play: 3546 | Frame: 50
Game: 2018090906 | Play: 3546 | Frame: 51
Game: 2018090906 | Play: 3546 | Frame: 52
Game: 2018090906 | Play: 3546 | Frame: 53
Game: 2018090906 | Play: 3546 | Frame: 54
Game: 2018090906 | Play: 3570 | Frame: 1
Game: 2018090906 | Play: 3570 | Frame: 2
Game: 2018090906 | Play: 3570 | Frame: 3
Game: 2018090906 | Play: 3570 | Frame: 4
Game: 2018090906 | Play: 3570 | Frame: 5
Game: 2018090906 | Play: 3570 | Frame: 6
Game: 2018090906 | Play: 3570 | Frame: 7
Game: 2018090906 | Play: 3570 | Frame: 8
Game: 2018090906 | Play: 3570 | Frame: 9
Game: 2018090906 | Play: 3570 | Frame: 10
G

Game: 2018090906 | Play: 3708 | Frame: 26
Game: 2018090906 | Play: 3708 | Frame: 27
Game: 2018090906 | Play: 3708 | Frame: 28
Game: 2018090906 | Play: 3708 | Frame: 29
Game: 2018090906 | Play: 3708 | Frame: 30
Game: 2018090906 | Play: 3708 | Frame: 31
Game: 2018090906 | Play: 3708 | Frame: 32
Game: 2018090906 | Play: 3708 | Frame: 33
Game: 2018090906 | Play: 3708 | Frame: 34
Game: 2018090906 | Play: 3708 | Frame: 35
Game: 2018090906 | Play: 3708 | Frame: 36
Game: 2018090906 | Play: 3708 | Frame: 37
Game: 2018090906 | Play: 3708 | Frame: 38
Game: 2018090906 | Play: 3708 | Frame: 39
Game: 2018090906 | Play: 3708 | Frame: 40
Game: 2018090906 | Play: 3708 | Frame: 41
Game: 2018090906 | Play: 3708 | Frame: 42
Game: 2018090906 | Play: 3708 | Frame: 43
Game: 2018090906 | Play: 3708 | Frame: 44
Game: 2018090906 | Play: 3708 | Frame: 45
Game: 2018090906 | Play: 3708 | Frame: 46
Game: 2018090906 | Play: 3708 | Frame: 47
Game: 2018090906 | Play: 3708 | Frame: 48
Game: 2018090906 | Play: 3708 | Fr

Game: 2018090906 | Play: 3808 | Frame: 1
Game: 2018090906 | Play: 3808 | Frame: 2
Game: 2018090906 | Play: 3808 | Frame: 3
Game: 2018090906 | Play: 3808 | Frame: 4
Game: 2018090906 | Play: 3808 | Frame: 5
Game: 2018090906 | Play: 3808 | Frame: 6
Game: 2018090906 | Play: 3808 | Frame: 7
Game: 2018090906 | Play: 3808 | Frame: 8
Game: 2018090906 | Play: 3808 | Frame: 9
Game: 2018090906 | Play: 3808 | Frame: 10
Game: 2018090906 | Play: 3808 | Frame: 11
Game: 2018090906 | Play: 3808 | Frame: 12
Game: 2018090906 | Play: 3808 | Frame: 13
Game: 2018090906 | Play: 3808 | Frame: 14
Game: 2018090906 | Play: 3808 | Frame: 15
Game: 2018090906 | Play: 3808 | Frame: 16
Game: 2018090906 | Play: 3808 | Frame: 17
Game: 2018090906 | Play: 3808 | Frame: 18
Game: 2018090906 | Play: 3808 | Frame: 19
Game: 2018090906 | Play: 3808 | Frame: 20
Game: 2018090906 | Play: 3808 | Frame: 21
Game: 2018090906 | Play: 3808 | Frame: 22
Game: 2018090906 | Play: 3808 | Frame: 23
Game: 2018090906 | Play: 3808 | Frame: 24
G

KeyboardInterrupt: 

In [164]:
temp

gameId  playId  frameId   displayName  closestDefender  \
0   2018090600      75        1   Julio Jones      Jalen Mills   
0   2018090600      75        1  Mohamed Sanu  Malcolm Jenkins   
0   2018090600      75        2   Julio Jones      Jalen Mills   
0   2018090600      75        2  Mohamed Sanu  Malcolm Jenkins   
0   2018090600      75        3   Julio Jones      Jalen Mills   
..         ...     ...      ...           ...              ...   
0   2018090600      75       57  Mohamed Sanu       Nate Gerry   
0   2018090600      75       58   Julio Jones  Malcolm Jenkins   
0   2018090600      75       58  Mohamed Sanu       Nate Gerry   
0   2018090600      75       59   Julio Jones  Malcolm Jenkins   
0   2018090600      75       59  Mohamed Sanu       Nate Gerry   

    closestDefenderDistance  
0                  7.380474  
0                  1.916899  
0                  7.381389  
0                  1.916899  
0                  7.371431  
..                      ...  
0                  2.205130  
0                  0.860523  
0                  2.018960  
0                  1.136002  
0                  1.842200  

[118 rows x 6 columns]

In [206]:
fbf_df_sample

time      x      y     s     a   dis       o     dir  \
0    2018-09-07T01:07:14.599Z  91.73  26.67  0.00  0.01  0.02  289.57  240.93   
1    2018-09-07T01:07:14.599Z  88.89  36.47  0.01  0.01  0.01  105.63   66.66   
2    2018-09-07T01:07:14.599Z  91.35  44.16  0.02  0.03  0.01  290.45   16.86   
3    2018-09-07T01:07:14.599Z  86.31  22.01  0.09  0.42  0.01   70.12  168.91   
4    2018-09-07T01:07:14.599Z  90.78  36.15  0.00  0.00  0.00  257.61  193.97   
..                        ...    ...    ...   ...   ...   ...     ...     ...   
821  2018-09-07T01:07:20.400Z  76.75  52.42  3.86  3.56  0.39   55.22   22.42   
822  2018-09-07T01:07:20.400Z  77.41   9.59  3.44  5.45  0.38   96.53  182.50   
823  2018-09-07T01:07:20.400Z  78.06  38.54  5.47  3.04  0.56  336.15  338.39   
824  2018-09-07T01:07:20.400Z  84.31   5.74  3.78  4.31  0.38   39.11  246.16   
825  2018-09-07T01:07:20.400Z  79.37  54.50  5.35  5.82  0.51     NaN     NaN   

    event      nflId      displayName  jerseyNumber position  frameId team  \
0    None      310.0        Matt Ryan           2.0       QB        1  ATL   
1    None    79848.0  Malcolm Jenkins          27.0       SS        1  PHI   
2    None  2495454.0      Julio Jones          11.0       WR        1  ATL   
3    None  2495613.0     Corey Graham          24.0       FS        1  PHI   
4    None  2533040.0     Mohamed Sanu          12.0       WR        1  ATL   
..    ...        ...              ...           ...      ...      ...  ...   
821  None  2555383.0      Jalen Mills          31.0       CB       59  PHI   
822  None  2555415.0    Austin Hooper          81.0       TE       59  ATL   
823  None  2558175.0       Nate Gerry          47.0       LB       59  PHI   
824  None  2559033.0      Ricky Ortiz          30.0       FB       59  ATL   
825  None        NaN         Football           NaN      NaN       59  ATL   

         gameId  playId playDirection  route week homeTeamAbbr  \
0    2018090600      75          left    NaN    1          PHI   
1    2018090600      75          left    NaN    1          PHI   
2    2018090600      75          left  HITCH    1          PHI   
3    2018090600      75          left    NaN    1          PHI   
4    2018090600      75          left  HITCH    1          PHI   
..          ...     ...           ...    ...  ...          ...   
821  2018090600      75          left    NaN    1          PHI   
822  2018090600      75          left    OUT    1          PHI   
823  2018090600      75          left    NaN    1          PHI   
824  2018090600      75          left   FLAT    1          PHI   
825  2018090600      75          left    NaN    1          PHI   

    visitorTeamAbbr possessionTeam  inPoss  
0               ATL            ATL       1  
1               ATL            ATL       0  
2               ATL            ATL       1  
3               ATL            ATL       0  
4               ATL            ATL       1  
..              ...            ...     ...  
821             ATL            ATL       0  
822             ATL            ATL       1  
823             ATL            ATL       0  
824             ATL            ATL       1  
825             ATL            ATL       1  

[826 rows x 24 columns]